In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, text
from smartsheet_dataframe import get_as_df

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet

In [3]:
# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

In [4]:
fha_df.head()

,row_id,parent_id,TICKET_ID,Date Received,Intake Date,Requested Harvest,Weeks,Assigned To,Scanned Completed,Snipping Complete,...,Tier 2 QA/QC Completed,GA Expected Removals SS Updated,Update Compiled Mill Ticket File,Update AHWP File,HWP Crediting Assessment Completed?,Final Havest Report Sent,Final Closing Check,Percent of total requested harvest completed,Comments,COMPLETED
0,619018378405764,,WAR191214TT01,2021-03-16,2021-03-16,33056_1,21.0,Unknown,True,True,...,True,True,True,True,,True,True,0.9914,,
1,5122618005776260,619018378405764,WAR191214TT01,2021-03-16,2021-03-16,33056_1,22.0,Unknown,True,True,...,True,True,True,True,,True,True,0.9914,,
2,2870818192091012,619018378405764,WAR191214TT01,2021-03-26,2021-03-26,33056_1,23.0,Unknown,True,True,...,True,True,True,True,,True,True,0.9914,,
3,7374417819461508,619018378405764,WAR191214TT01,2021-03-30,2021-03-30,33056_1,24.0,Unknown,True,True,...,True,True,True,True,,True,True,1.0089,,
4,1744918285248388,619018378405764,WAR191214TT01,2021-04-12,2021-04-12,33056_1,25.0,Unknown,True,True,...,True,True,True,True,,True,True,1.0741,,


In [5]:
fha_df.columns

Index(['row_id', 'parent_id', 'TICKET_ID', 'Date Received', 'Intake Date',
       'Requested Harvest', 'Weeks', 'Assigned To', 'Scanned Completed',
       'Snipping Complete', 'Data Entry Complete', 'Tier 1 QA/QC Completed',
       'Tier 2 QA/QC Completed', 'GA Expected Removals SS Updated',
       'Update Compiled Mill Ticket File', 'Update AHWP File',
       'HWP Crediting Assessment Completed?', 'Final Havest Report Sent',
       'Final Closing Check', 'Percent of total requested harvest completed',
       'Comments', 'COMPLETED'],
      dtype='object')

In [6]:
# Get only the columns that I need in the db
fha_df = fha_df[['TICKET_ID', 'Date Received', 'Intake Date', 'Weeks', 'Assigned To']]
fha_df = fha_df.rename(columns={'TICKET_ID': 'ticketid', 'Date Received': 'received_date', 'Intake Date': 'intake_date', 'Weeks': 'package_week', 'Assigned To': 'assigned_to'})
fha_df.head()

,ticketid,received_date,intake_date,package_week,assigned_to
0,WAR191214TT01,2021-03-16,2021-03-16,21.0,Unknown
1,WAR191214TT01,2021-03-16,2021-03-16,22.0,Unknown
2,WAR191214TT01,2021-03-26,2021-03-26,23.0,Unknown
3,WAR191214TT01,2021-03-30,2021-03-30,24.0,Unknown
4,WAR191214TT01,2021-04-12,2021-04-12,25.0,Unknown


In [7]:
fha_df['assigned_to'].value_counts()

assigned_to
Unknown    426
Franz      374
Lee         94
Kelsey      88
Ruben       66
Lydia       49
Rob         46
TBD         20
Matt         3
Bily         3
Name: count, dtype: int64

In [8]:
# Change assigned_to value to mapped foreign key (from 'ticket_entry_employees' table)
ticket_employee_map = {"Franz": 1,
                       "Kelsey": 2,
                       "Bily": 3,
                       "Rob": 4,
                       "Lee": 5,
                       "Lydia": 6,
                       "Ruben": 7,
                       "TBD": 8,
                       "Unknown": 9,
                       "Matt": 10}
fha_df['assigned_to'] = fha_df['assigned_to'].map(ticket_employee_map)

In [9]:
fha_df['package_week'] = fha_df['package_week'].astype(int)

In [21]:
# Let's make sure the dates work
fha_df['intake_date'] = pd.to_datetime(fha_df['intake_date'])
fha_df['received_date'] = pd.to_datetime(fha_df['received_date'], )
fha_df['intake_date'] = fha_df['intake_date'].fillna('1900-01-01')
fha_df['received_date'] = fha_df['received_date'].fillna('1900-01-01')
fha_df

,ticketid,received_date,intake_date,package_week,assigned_to
0,WAR191214TT01,2021-03-16,2021-03-16,21,9
1,WAR191214TT01,2021-03-16,2021-03-16,22,9
2,WAR191214TT01,2021-03-26,2021-03-26,23,9
3,WAR191214TT01,2021-03-30,2021-03-30,24,9
4,WAR191214TT01,2021-04-12,2021-04-12,25,9
...,...,...,...,...,...
1164,WAR230908TH04,1900-01-01,1900-01-01,2,8
1165,WAR230908TH04,1900-01-01,1900-01-01,3,8
1166,WAR230908TH04,1900-01-01,1900-01-01,4,8
1167,WAR230908TH04,1900-01-01,1900-01-01,5,8


In [22]:
# setup db connection
db_host = 'localhost'
db_name = 'HarvestManagementTrial'
db_user = 'postgres'
db_password = 'root'

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}'


In [23]:
# Create the SQLAlchemy engine
engine = create_engine(connection_string)

In [24]:
fha_df.to_sql('fha_table', engine, if_exists='append', index=False)

169

In [25]:
engine.dispose()